In [1]:
import pandas as pd
import numpy as np
from time import time
import json
from sqlalchemy import create_engine
import re
import datetime
import lifetimes
#import pymc3 as pm

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
plt.style.use('seaborn-whitegrid')
import numpy as np
#import plotly.plotly as py
#import plotly.graph_objs as go
#from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#cufflinks.go_offline(connected=True)
#init_notebook_mode(connected=True)
#import lifelines

#import plotly
#plotly.tools.set_config_file(sharing='public')

In [2]:
# Set up the SQL connection
dbname = 'redshift'
url = 'panamera-ods.cnsuxis6zqxr.us-west-2.redshift.amazonaws.com:5439/panameraods'
engine = create_engine("postgresql://" + 'lucasdresl' + ":" + 'cQsY801a76x82x62aWDT' + "@" + url)

# ANDROID

## MEA: Profile completion on the onboarding

-  How many users are completing something
-  How many complete the entire flow 
-  How many user are dropping and in which step
-  This users which are completing something are usually users which are more retain?
-  How to measure impact? replies increases? listing increases? people are more confident so there is more interactions? what 

In [10]:
query = """

WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and 
)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' THEN session_long ELSE NULL END) AS profile_completion_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'phone_validation' THEN session_long ELSE NULL END) AS puso_al_menos_phone,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'email_validation' THEN session_long ELSE NULL END) AS puso_al_menos_email,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS has_picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS valid_name,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS social_account
FROM ods.panameraolx_mea_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
;

  
"""

query_has_about = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_about'
)
SELECT
country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS profile_completion_show_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about
FROM ods.panameraolx_mea_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
;

"""


query_has_picture = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_picture'
)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS profile_completion_show_picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS has_picture
FROM ods.panameraolx_mea_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
;

"""

query_valid_social_account = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_social_account'
)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS profile_completion_show_social_account,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS valid_social_account
FROM ods.panameraolx_mea_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
;

"""

query_real_name = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_name'
)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS profile_completion_name_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS valid_name
FROM ods.panameraolx_mea_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
;

"""


query_valid_email = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month

WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_email' 
)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_email' THEN session_long ELSE NULL END) AS profile_completion_email_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step='email_validation' THEN session_long ELSE NULL END) AS valid_email
FROM ods.panameraolx_mea_hydra_ninja_android_last_month

JOIN verified USING (session)
group by 1
;

"""


query_valid_phone = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_phone'

)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_phone' THEN session_long ELSE NULL END) AS profile_completion_phone_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step='phone_validation' THEN session_long ELSE NULL END) AS valid_phone
FROM ods.panameraolx_mea_hydra_ninja_android_last_month

JOIN verified USING (session)
group by 1
;

"""


In [11]:
df_mea = pd.read_sql_query(query.replace('%', '%%'), engine)
df_mea_has_about = pd.read_sql_query(query_has_about.replace('%', '%%'), engine)
df_mea_has_picture = pd.read_sql_query(query_has_picture.replace('%', '%%'), engine)
df_mea_valid_social_account = pd.read_sql_query(query_valid_social_account.replace('%', '%%'), engine)
df_mea_real_name = pd.read_sql_query(query_real_name.replace('%', '%%'), engine)
df_mea_valid_email = pd.read_sql_query(query_valid_email.replace('%', '%%'), engine)
df_mea_valid_phone = pd.read_sql_query(query_valid_phone.replace('%', '%%'), engine)

In [56]:
#df_mea.sort_values(['date_event_nk']).tail()

-  PICTURE

In [13]:
df_mea_has_picture['has_picture_conversion'] = df_mea_has_picture['has_picture'] / df_mea_has_picture['profile_completion_show_picture'] 
df_mea_has_picture.head()

,country_sk,profile_completion_show_picture,has_picture,has_picture_conversion
0,olx|mea|ug,171,0,0.0
1,olx|mea|gh,586,0,0.0
2,olx|mea|za,26372,0,0.0
3,olx|mea|ng,6,0,0.0
4,olx|mea|tz,5,0,0.0


- NAME

In [14]:
df_mea_real_name['valid_name_conversion'] = df_mea_real_name['valid_name'] / df_mea_real_name['profile_completion_name_show'] 
df_mea_real_name.head()

,country_sk,profile_completion_name_show,valid_name,valid_name_conversion
0,olx|mea|za,16041,14619,0.911352
1,olx|mea|ug,105,92,0.876190
2,olx|mea|gh,365,330,0.904110
3,olx|mea|ng,3,2,0.666667
4,olx|mea|pk,83105,71025,0.854642


- HAS ABOUT

In [15]:
df_mea_has_about['has_about_conversion'] = df_mea_has_about['has_about'] / df_mea_has_about['profile_completion_show_about'] 
df_mea_has_about.head()

,country_sk,profile_completion_show_about,has_about,has_about_conversion
0,olx|mea|za,33040,6466,0.195702
1,olx|mea|ug,181,67,0.370166
2,olx|mea|ng,7,3,0.428571
3,olx|mea|gh,756,194,0.256614
4,olx|mea|pk,307050,40366,0.131464


- HAS EMAIL

In [16]:
df_mea_valid_email['has_valid_email_conversion'] = df_mea_valid_email['valid_email'] / df_mea_valid_email['profile_completion_email_show'] 
df_mea_valid_email.head()

,country_sk,profile_completion_email_show,valid_email,has_valid_email_conversion
0,olx|mea|za,18320,3989,0.217740
1,olx|mea|gh,466,108,0.231760
2,olx|mea|pk,94314,12931,0.137106
3,olx|mea|ng,4,2,0.500000
4,olx|mea|ug,113,25,0.221239


- HAS PHONE

In [17]:
df_mea_valid_phone['has_valid_phone_conversion'] = df_mea_valid_phone['valid_phone'] / df_mea_valid_phone['profile_completion_phone_show'] 
phone = df_mea_valid_phone['has_valid_phone_conversion'] 
df_mea_valid_phone.head()

,country_sk,profile_completion_phone_show,valid_phone,has_valid_phone_conversion
0,olx|mea|ug,175,73,0.417143
1,olx|mea|za,37925,12914,0.340514
2,olx|mea|gh,823,329,0.399757
3,olx|mea|tz,10,4,0.400000
4,olx|mea|ng,3,2,0.666667


- HAS SOCIAL ACCOUNT

In [18]:
df_mea_valid_social_account['social_account_conversion'] = df_mea_valid_social_account['valid_social_account'] / df_mea_valid_social_account['profile_completion_show_social_account'] 
social_account = df_mea_valid_social_account['social_account_conversion']
df_mea_valid_social_account.head()

,country_sk,profile_completion_show_social_account,valid_social_account,social_account_conversion
0,olx|mea|za,12012,0,0.0
1,olx|mea|ug,96,0,0.0
2,olx|mea|gh,311,0,0.0
3,olx|mea|ng,4,0,0.0
4,olx|mea|pk,101530,0,0.0


## MEA: Profile completion on the returning bar

In [19]:
query = """

WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar'
)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' THEN session_long ELSE NULL END) AS profile_completion_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'phone_validation' THEN session_long ELSE NULL END) AS puso_al_menos_phone,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'email_validation' THEN session_long ELSE NULL END) AS puso_al_menos_email,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS has_picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS valid_name,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS social_account
FROM ods.panameraolx_mea_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

  
"""

query_has_about = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'has_about'
)
SELECT
date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS profile_completion_show_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about
FROM ods.panameraolx_mea_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""


query_has_picture = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'has_picture'
)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS profile_completion_show_picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS has_picture
FROM ods.panameraolx_mea_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""

query_valid_social_account = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_social_account'
)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS profile_completion_show_social_account,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS valid_social_account
FROM ods.panameraolx_mea_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""

query_real_name = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_name'
)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS profile_completion_name_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS valid_name
FROM ods.panameraolx_mea_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""


query_valid_email = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month

WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_email' 
)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_email' THEN session_long ELSE NULL END) AS profile_completion_email_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step='email_validation' THEN session_long ELSE NULL END) AS valid_email
FROM ods.panameraolx_mea_hydra_ninja_android_last_month

JOIN verified USING (session)
group by 1
order by 1
;

"""


query_valid_phone = """
WITH verified AS (
    SELECT session
from ods.panameraolx_mea_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_phone'

)
SELECT
  country_sk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_phone' THEN session_long ELSE NULL END) AS profile_completion_phone_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step='phone_validation' THEN session_long ELSE NULL END) AS valid_phone
FROM ods.panameraolx_mea_hydra_ninja_android_last_month

JOIN verified USING (session)
group by 1
order by 1
;

"""


In [20]:
df_mea = pd.read_sql_query(query.replace('%', '%%'), engine)
df_mea_has_about = pd.read_sql_query(query_has_about.replace('%', '%%'), engine)
df_mea_has_picture = pd.read_sql_query(query_has_picture.replace('%', '%%'), engine)
df_mea_valid_social_account = pd.read_sql_query(query_valid_social_account.replace('%', '%%'), engine)
df_mea_real_name = pd.read_sql_query(query_real_name.replace('%', '%%'), engine)
df_mea_valid_email = pd.read_sql_query(query_valid_email.replace('%', '%%'), engine)
df_mea_valid_phone = pd.read_sql_query(query_valid_phone.replace('%', '%%'), engine)

In [21]:
#df_mea.sort_values(['date_event_nk']).tail()

-  PICTURE

In [22]:
df_mea_has_picture['has_picture_conversion'] = df_mea_has_picture['has_picture'] / df_mea_has_picture['profile_completion_show_picture'] 
df_mea_has_picture.head()

,country_sk,profile_completion_show_picture,has_picture,has_picture_conversion
0,olx|mea|gh,68,5,0.073529
1,olx|mea|ng,2,0,0.000000
2,olx|mea|pk,43544,10,0.000230
3,olx|mea|ug,11,0,0.000000
4,olx|mea|za,3523,348,0.098779


- NAME

In [23]:
df_mea_real_name['valid_name_conversion'] = df_mea_real_name['valid_name'] / df_mea_real_name['profile_completion_name_show'] 
df_mea_real_name.head()

,country_sk,profile_completion_name_show,valid_name,valid_name_conversion
0,olx|mea|gh,27,17,0.629630
1,olx|mea|ng,1,0,0.000000
2,olx|mea|pk,10946,6846,0.625434
3,olx|mea|ug,2,2,1.000000
4,olx|mea|za,1083,742,0.685134


- HAS ABOUT

In [24]:
df_mea_has_about['has_about_conversion'] = df_mea_has_about['has_about'] / df_mea_has_about['profile_completion_show_about'] 
df_mea_has_about.head()

,date_event_nk,profile_completion_show_about,has_about,has_about_conversion
0,2019-06-01,646,207,0.320433
1,2019-06-02,738,232,0.314363
2,2019-06-03,711,233,0.327707
3,2019-06-04,558,167,0.299283
4,2019-06-05,407,155,0.380835


- HAS EMAIL

In [25]:
df_mea_valid_email['has_valid_email_conversion'] = df_mea_valid_email['valid_email'] / df_mea_valid_email['profile_completion_email_show'] 
df_mea_valid_email.tail()

,country_sk,profile_completion_email_show,valid_email,has_valid_email_conversion
1,olx|mea|ke,1,0,0.000000
2,olx|mea|ng,3,0,0.000000
3,olx|mea|pk,17069,1949,0.114184
4,olx|mea|ug,20,2,0.100000
5,olx|mea|za,2791,488,0.174848


- HAS PHONE

In [26]:
df_mea_valid_phone['has_valid_phone_conversion'] = df_mea_valid_phone['valid_phone'] / df_mea_valid_phone['profile_completion_phone_show'] 
phone = df_mea_valid_phone['has_valid_phone_conversion'] 
df_mea_valid_phone.tail()

,country_sk,profile_completion_phone_show,valid_phone,has_valid_phone_conversion
0,olx|mea|gh,79,14,0.177215
1,olx|mea|ng,3,0,0.000000
2,olx|mea|pk,20516,3194,0.155683
3,olx|mea|ug,9,1,0.111111
4,olx|mea|za,4004,509,0.127123


- HAS SOCIAL ACCOUNT

In [27]:
df_mea_valid_social_account['social_account_conversion'] = df_mea_valid_social_account['valid_social_account'] / df_mea_valid_social_account['profile_completion_show_social_account'] 
social_account = df_mea_valid_social_account['social_account_conversion']
df_mea_valid_social_account.head()

,country_sk,profile_completion_show_social_account,valid_social_account,social_account_conversion
0,olx|mea|gh,54,0,0.0
1,olx|mea|ng,5,0,0.0
2,olx|mea|pk,18613,0,0.0
3,olx|mea|ug,7,0,0.0
4,olx|mea|za,2065,0,0.0


## INDIA: Profile completion on the onboarding

In [41]:
query = """

WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and date_event_nk = '2019-07-30'
     and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' THEN session_long ELSE NULL END) AS profile_completion_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'phone_validation' THEN session_long ELSE NULL END) AS puso_al_menos_phone,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'email_validation' THEN session_long ELSE NULL END) AS puso_al_menos_email,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS has_picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS valid_name,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS social_account
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

  
"""

query_has_about = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_about'
 and date_event_nk = '2019-07-30' and app_version = '13.19.02'
)
SELECT
date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS profile_completion_show_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""


query_has_picture = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_picture'
and date_event_nk = '2019-07-30' and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS profile_completion_show_picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'computer' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS has_picture
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""

query_valid_social_account = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_social_account'
and date_event_nk = '2019-07-30' and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS profile_completion_show_social_account,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS valid_social_account
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""

query_real_name = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_name'
and date_event_nk = '2019-07-30' and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS profile_completion_name_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS valid_name
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""


query_valid_email = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month

WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_email' 
and date_event_nk = '2019-07-30' and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_email' THEN session_long ELSE NULL END) AS profile_completion_email_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step='email_validation' THEN session_long ELSE NULL END) AS valid_email
FROM ods.panameraolx_asia_hydra_ninja_android_last_month

JOIN verified USING (session)
group by 1
order by 1
;

"""


query_valid_phone = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_phone'
and date_event_nk = '2019-07-30' and app_version = '13.19.02'

)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_phone' THEN session_long ELSE NULL END) AS profile_completion_phone_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step='phone_validation' THEN session_long ELSE NULL END) AS valid_phone
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""


In [42]:
df_asia_all = pd.read_sql_query(query.replace('%', '%%'), engine)
df_asia_has_about = pd.read_sql_query(query_has_about.replace('%', '%%'), engine)
df_asia_has_picture = pd.read_sql_query(query_has_picture.replace('%', '%%'), engine)
df_asia_valid_social_account = pd.read_sql_query(query_valid_social_account.replace('%', '%%'), engine)
df_asia_real_name = pd.read_sql_query(query_real_name.replace('%', '%%'), engine)
df_asia_valid_email = pd.read_sql_query(query_valid_email.replace('%', '%%'), engine)
df_asia_valid_phone = pd.read_sql_query(query_valid_phone.replace('%', '%%'), engine)

-  PICTURE

In [43]:
df_asia_has_picture['has_picture_conversion'] = df_asia_has_picture['has_picture'] / df_asia_has_picture['profile_completion_show_picture'] 
df_asia_has_picture.tail()

,date_event_nk,profile_completion_show_picture,has_picture,has_picture_conversion
0,2019-07-29,0,0,NaN
1,2019-07-30,81906,9243,0.112849
2,2019-07-31,0,1,inf
3,2019-08-01,0,0,NaN


- NAME

In [44]:
df_asia_real_name['valid_name_conversion'] = df_asia_real_name['valid_name'] / df_asia_real_name['profile_completion_name_show'] 
df_asia_real_name.tail()

,date_event_nk,profile_completion_name_show,valid_name,valid_name_conversion
0,2019-07-29,0,0,NaN
1,2019-07-30,23405,20224,0.864089
2,2019-07-31,0,2,inf
3,2019-08-01,0,0,NaN


- HAS ABOUT

In [45]:
df_asia_has_about['has_about_conversion'] = df_asia_has_about['has_about'] / df_asia_has_about['profile_completion_show_about'] 
df_asia_has_about.tail()

,date_event_nk,profile_completion_show_about,has_about,has_about_conversion
1,2019-07-28,0,0,NaN
2,2019-07-29,0,0,NaN
3,2019-07-30,98913,12122,0.122552
4,2019-07-31,0,3,inf
5,2019-08-01,0,0,NaN


- HAS EMAIL

In [46]:
df_asia_valid_email['has_valid_email_conversion'] = df_asia_valid_email['valid_email'] / df_asia_valid_email['profile_completion_email_show'] 
df_asia_valid_email.tail()

,date_event_nk,profile_completion_email_show,valid_email,has_valid_email_conversion
0,2019-07-29,6,0,0.000000
1,2019-07-30,57485,9419,0.163851
2,2019-07-31,5,3,0.600000
3,2019-08-01,0,0,NaN


- HAS PHONE

In [47]:
df_asia_valid_phone['has_valid_phone_conversion'] = df_asia_valid_phone['valid_phone'] / df_asia_valid_phone['profile_completion_phone_show'] 
phone = df_asia_valid_phone['has_valid_phone_conversion'] 
df_asia_valid_phone.tail()

,date_event_nk,profile_completion_phone_show,valid_phone,has_valid_phone_conversion
1,2019-07-28,2,0,0.000000
2,2019-07-29,21,0,0.000000
3,2019-07-30,71549,28170,0.393716
4,2019-07-31,14,9,0.642857
5,2019-08-01,0,0,NaN


- HAS SOCIAL ACCOUNT

In [48]:
df_asia_valid_social_account['social_account_conversion'] = df_asia_valid_social_account['valid_social_account'] / df_asia_valid_social_account['profile_completion_show_social_account'] 
social_account = df_asia_valid_social_account['social_account_conversion']
df_asia_valid_social_account.tail()

,date_event_nk,profile_completion_show_social_account,valid_social_account,social_account_conversion
0,2019-07-29,0,0,NaN
1,2019-07-30,29691,0,0.0
2,2019-07-31,0,0,NaN
3,2019-08-01,0,0,NaN


## INDIA: Profile completion on the returning bar

In [30]:
query = """

WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and date_event_nk = '2019-07-30'
  and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' THEN session_long ELSE NULL END) AS profile_completion_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'phone_validation' THEN session_long ELSE NULL END) AS puso_al_menos_phone,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'email_validation' THEN session_long ELSE NULL END) AS puso_al_menos_email,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS has_picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS valid_name,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS social_account
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

  
"""

query_has_about = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'has_about'
and date_event_nk = '2019-07-30' and app_version = '13.19.02'
)
SELECT
date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS profile_completion_show_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""


query_has_picture = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'has_picture'
and date_event_nk = '2019-07-30' and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS profile_completion_show_picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'computer' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS has_picture
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""

query_valid_social_account = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_social_account'
and date_event_nk = '2019-07-30' and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS profile_completion_show_social_account,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS valid_social_account
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""

query_real_name = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_name'
and date_event_nk = '2019-07-30' and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS profile_completion_name_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS valid_name
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1
;

"""


query_valid_email = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month

WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_email' 
and date_event_nk = '2019-07-30' and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_email' THEN session_long ELSE NULL END) AS profile_completion_email_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step='email_validation' THEN session_long ELSE NULL END) AS valid_email
FROM ods.panameraolx_asia_hydra_ninja_android_last_month

JOIN verified USING (session)
group by 1
order by 1
;

"""


query_valid_phone = """
WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_phone'
and date_event_nk = '2019-07-30' and app_version = '13.19.02'

)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'profile_bar' and flow_step = 'valid_phone' THEN session_long ELSE NULL END) AS profile_completion_phone_show,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_action' and chosen_option = 'continue' and flow_step='phone_validation' THEN session_long ELSE NULL END) AS valid_phone
FROM ods.panameraolx_asia_hydra_ninja_android_last_month

JOIN verified USING (session)
group by 1
order by 1
;

"""


In [31]:
df_asia_all = pd.read_sql_query(query.replace('%', '%%'), engine)
df_asia_has_about = pd.read_sql_query(query_has_about.replace('%', '%%'), engine)
df_asia_has_picture = pd.read_sql_query(query_has_picture.replace('%', '%%'), engine)
df_asia_valid_social_account = pd.read_sql_query(query_valid_social_account.replace('%', '%%'), engine)
df_asia_real_name = pd.read_sql_query(query_real_name.replace('%', '%%'), engine)
df_asia_valid_email = pd.read_sql_query(query_valid_email.replace('%', '%%'), engine)
df_asia_valid_phone = pd.read_sql_query(query_valid_phone.replace('%', '%%'), engine)

In [32]:
df_asia_all.sort_values(['date_event_nk']).tail()

,date_event_nk,profile_completion_show,has_about,puso_al_menos_phone,puso_al_menos_email,has_picture,valid_name,social_account
0,2019-07-29,11,2,3,1,0,7,0
1,2019-07-30,13726,2369,1233,1220,0,1513,0
2,2019-07-31,14,3,0,3,0,0,0
3,2019-08-01,0,0,0,0,0,0,0


-  PICTURE

In [33]:
df_asia_has_picture['has_picture_conversion'] = df_asia_has_picture['has_picture'] / df_asia_has_picture['profile_completion_show_picture'] 
df_asia_has_picture.tail()

,date_event_nk,profile_completion_show_picture,has_picture,has_picture_conversion
0,2019-07-29,3,1,0.333333
1,2019-07-30,7270,1731,0.238102
2,2019-07-31,2,3,1.500000
3,2019-08-01,0,0,NaN


- NAME

In [34]:
df_asia_real_name['valid_name_conversion'] = df_asia_real_name['valid_name'] / df_asia_real_name['profile_completion_name_show'] 
df_asia_real_name.tail()

,date_event_nk,profile_completion_name_show,valid_name,valid_name_conversion
0,2019-07-29,0,0,NaN
1,2019-07-30,1030,642,0.623301
2,2019-07-31,0,0,NaN
3,2019-08-01,0,0,NaN


- HAS ABOUT

In [35]:
df_asia_has_about['has_about_conversion'] = df_asia_has_about['has_about'] / df_asia_has_about['profile_completion_show_about'] 
df_asia_has_about.tail()

,date_event_nk,profile_completion_show_about,has_about,has_about_conversion
0,2019-07-29,2,0,0.00000
1,2019-07-30,7855,2221,0.28275
2,2019-07-31,1,1,1.00000


- HAS EMAIL

In [36]:
df_asia_valid_email['has_valid_email_conversion'] = df_asia_valid_email['valid_email'] / df_asia_valid_email['profile_completion_email_show'] 
df_asia_valid_email.tail()

,date_event_nk,profile_completion_email_show,valid_email,has_valid_email_conversion
0,2019-07-29,3,0,0.000000
1,2019-07-30,5004,944,0.188649
2,2019-07-31,3,2,0.666667
3,2019-08-01,0,0,NaN


- HAS PHONE

In [37]:
df_asia_valid_phone['has_valid_phone_conversion'] = df_asia_valid_phone['valid_phone'] / df_asia_valid_phone['profile_completion_phone_show'] 
phone = df_asia_valid_phone['has_valid_phone_conversion'] 
df_asia_valid_phone.tail()

,date_event_nk,profile_completion_phone_show,valid_phone,has_valid_phone_conversion
0,2019-07-29,3,0,0.000000
1,2019-07-30,3030,679,0.224092
2,2019-07-31,4,0,0.000000


- HAS SOCIAL ACCOUNT

In [38]:
df_asia_valid_social_account['social_account_conversion'] = df_asia_valid_social_account['valid_social_account'] / df_asia_valid_social_account['profile_completion_show_social_account'] 
social_account = df_asia_valid_social_account['social_account_conversion']
df_asia_valid_social_account.tail()

,date_event_nk,profile_completion_show_social_account,valid_social_account,social_account_conversion
0,2019-07-29,0,0,NaN
1,2019-07-30,2665,0,0.0
2,2019-07-31,1,0,0.0


# Which login methods are showing the picture?

In [67]:
query_facebook = """


WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'login_sign_in_complete' and origin_nk = 'on_boarding' and login_method = 'facebook'
and date_event_nk in ('2019-07-30','2019-07-31') and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'login_sign_in_complete' and origin_nk = 'on_boarding' and login_method = 'facebook' THEN session_long ELSE NULL END) AS login_facebook,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS social_account,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS name,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_email' THEN session_long ELSE NULL END) AS email,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_phone' THEN session_long ELSE NULL END) AS phone
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1


"""


query_google = """


WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'login_sign_in_complete' and origin_nk = 'on_boarding' and login_method = 'google'
and date_event_nk in ('2019-07-30','2019-07-31') and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'login_sign_in_complete' and origin_nk = 'on_boarding' and login_method = 'google' THEN session_long ELSE NULL END) AS login_google,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS social_account,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS name,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_email' THEN session_long ELSE NULL END) AS email,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_phone' THEN session_long ELSE NULL END) AS phone
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1


"""

query_phone = """


WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'login_sign_in_complete' and origin_nk = 'on_boarding' and login_method = 'phone'
and date_event_nk in ('2019-07-30','2019-07-31') and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'login_sign_in_complete' and origin_nk = 'on_boarding' and login_method = 'phone' THEN session_long ELSE NULL END) AS login_phone,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS social_account,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS name,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_email' THEN session_long ELSE NULL END) AS email,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_phone' THEN session_long ELSE NULL END) AS phone
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1


"""

query_email = """


WITH verified AS (
    SELECT session
from ods.panameraolx_asia_hydra_ninja_android_last_month
WHERE trackevent = 'login_sign_in_complete' and origin_nk = 'on_boarding' and login_method = 'email'
and date_event_nk in ('2019-07-30','2019-07-31') and app_version = '13.19.02'
)
SELECT
  date_event_nk,
  COUNT(DISTINCT CASE WHEN trackevent = 'login_sign_in_complete' and origin_nk = 'on_boarding' and login_method = 'email' THEN session_long ELSE NULL END) AS login_email,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_about' THEN session_long ELSE NULL END) AS has_about,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'has_picture' THEN session_long ELSE NULL END) AS picture,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_social_account' THEN session_long ELSE NULL END) AS social_account,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_name' THEN session_long ELSE NULL END) AS name,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_email' THEN session_long ELSE NULL END) AS email,
  COUNT(DISTINCT CASE WHEN trackevent = 'profile_completion_show' and origin_nk = 'on_boarding' and flow_step = 'valid_phone' THEN session_long ELSE NULL END) AS phone
FROM ods.panameraolx_asia_hydra_ninja_android_last_month
JOIN verified USING (session)
group by 1
order by 1


"""


In [68]:
df_facebook = pd.read_sql_query(query_facebook.replace('%', '%%'), engine)
df_google = pd.read_sql_query(query_google.replace('%', '%%'), engine)
df_phone = pd.read_sql_query(query_phone.replace('%', '%%'), engine)
df_email = pd.read_sql_query(query_email.replace('%', '%%'), engine)

In [69]:
df_facebook.head()

,date_event_nk,login_facebook,has_about,picture,social_account,name,email,phone
0,2019-07-29,0,0,0,0,0,0,0
1,2019-07-30,38049,21280,17444,3,518,21059,25196
2,2019-07-31,37373,20812,17182,2,512,20662,24727
3,2019-08-01,0,7,0,0,0,9,6


In [70]:
df_google.head()

,date_event_nk,login_google,has_about,picture,social_account,name,email,phone
0,2019-07-25,0,0,0,0,0,0,0
1,2019-07-26,0,0,0,0,0,0,0
2,2019-07-27,0,0,0,0,0,0,0
3,2019-07-29,1,0,0,0,0,0,0
4,2019-07-30,65842,40529,20169,54,3343,312,43760


In [71]:
df_phone.head()

,date_event_nk,login_phone,has_about,picture,social_account,name,email,phone
0,2019-07-29,1,0,0,0,0,0,0
1,2019-07-30,55574,34381,41159,27753,19229,36065,31
2,2019-07-31,53961,32929,39729,26669,18660,34829,35
3,2019-08-01,0,24,18,18,12,33,0


In [72]:
df_email.head()

,date_event_nk,login_email,has_about,picture,social_account,name,email,phone
0,2019-07-29,0,0,0,0,0,0,0
1,2019-07-30,4748,2439,3193,1816,331,4,2537
2,2019-07-31,4597,2321,3097,1726,314,3,2417
3,2019-08-01,0,1,0,0,0,0,0


# Profile completion conversion for each login method

In [121]:

query = """

drop table if exists loggers;
select
  session_long,
  session,
  login_method
  into temp table loggers
   from ods.panameraolx_asia_hydra_ninja_android_last_month
where
     trackevent = 'login_sign_in_complete'
 and origin_nk  = 'on_boarding'
 and date_event_nk = '2019-07-30'
 and app_version = '13.19.02'
group by 1,2,3



;drop table if exists profile_completion_show_about;
select
 session_long,
 session
  into temp table profile_completion_show_about
  from ods.panameraolx_asia_hydra_ninja_android_last_month
where
  trackevent = 'profile_completion_show'
and origin_nk='on_boarding'
and flow_step='has_about'
and date_event_nk='2019-07-30'
and app_version = '13.19.02'
group by 1,2

;drop table if exists profile_completion_action_about;
select
 session_long,
 session
  into temp table profile_completion_action_about
  from ods.panameraolx_asia_hydra_ninja_android_last_month
where
  trackevent = 'profile_completion_action'
and origin_nk='on_boarding'
and flow_step='has_about'
and chosen_option='continue'
and date_event_nk='2019-07-30'
and app_version = '13.19.02'
group by 1,2


;drop table if exists profile_completion_show_picture;
select
 session_long,
 session
  into temp table profile_completion_show_picture
  from ods.panameraolx_asia_hydra_ninja_android_last_month
where
  trackevent = 'profile_completion_show'
and origin_nk='on_boarding'
and flow_step='has_picture'
and date_event_nk='2019-07-30'
and app_version = '13.19.02'
group by 1,2

;drop table if exists profile_completion_action_picture;
select
 session_long,
 session
  into temp table profile_completion_action_picture
  from ods.panameraolx_asia_hydra_ninja_android_last_month
where
  trackevent = 'profile_completion_action'
and origin_nk='on_boarding'
and flow_step='has_picture'
and date_event_nk='2019-07-30'
and chosen_option in ('computer' , 'continue')
and app_version = '13.19.02'
group by 1,2

;drop table if exists profile_completion_show_name;
select
 session_long,
 session
  into temp table profile_completion_show_name
  from ods.panameraolx_asia_hydra_ninja_android_last_month
where
  trackevent = 'profile_completion_show'
and origin_nk='on_boarding'
and flow_step='valid_name'
and date_event_nk='2019-07-30'
and app_version = '13.19.02'
group by 1,2

;drop table if exists profile_completion_action_name;
select
 session_long,
 session
  into temp table profile_completion_action_name
  from ods.panameraolx_asia_hydra_ninja_android_last_month
where
  trackevent = 'profile_completion_action'
and origin_nk='on_boarding'
and flow_step='valid_name'
and chosen_option='continue'
and date_event_nk='2019-07-30'
and app_version = '13.19.02'
group by 1,2


;drop table if exists profile_completion_show_email;
select
 session_long,
 session
  into temp table profile_completion_show_email
  from ods.panameraolx_asia_hydra_ninja_android_last_month
where
  trackevent = 'profile_completion_show'
and origin_nk='on_boarding'
and flow_step='valid_email'
and date_event_nk='2019-07-30'
and app_version = '13.19.02'
group by 1,2

;drop table if exists profile_completion_action_email;
select
 session_long,
 session
  into temp table profile_completion_action_email
  from ods.panameraolx_asia_hydra_ninja_android_last_month
where
  trackevent = 'profile_completion_action'
and origin_nk='on_boarding'
and flow_step='email_validation'
and chosen_option='continue'
and date_event_nk='2019-07-30'
and app_version = '13.19.02'
group by 1,2


;drop table if exists profile_completion_show_phone;
select
 session_long,
 session
  into temp table profile_completion_show_phone
  from ods.panameraolx_asia_hydra_ninja_android_last_month
where
  trackevent = 'profile_completion_show'
and origin_nk='on_boarding'
and flow_step='valid_phone'
and date_event_nk='2019-07-30'
and app_version = '13.19.02'
group by 1,2

;drop table if exists profile_completion_action_phone;
select
 session_long,
 session
  into temp table profile_completion_action_phone
  from ods.panameraolx_asia_hydra_ninja_android_last_month
where
  trackevent = 'profile_completion_action'
and origin_nk='on_boarding'
and flow_step='phone_validation'
and chosen_option='continue'
and date_event_nk='2019-07-30'
and app_version = '13.19.02'
group by 1,2;


select
    a.*,
    case when b.session is not null then 1 else 0 end as show_about,
    case when c.session is not null then 1 else 0 end as continue_about,
    case when d.session is not null then 1 else 0 end as show_name,
    case when e.session is not null then 1 else 0 end as continue_name,
    case when f.session is not null then 1 else 0 end as show_picture,
    case when g.session is not null then 1 else 0 end as continue_picture,
    case when h.session is not null then 1 else 0 end as show_phone,
    case when i.session is not null then 1 else 0 end as continue_phone,
    case when j.session is not null then 1 else 0 end as show_email,
    case when k.session is not null then 1 else 0 end as continue_email
  from loggers a
    left join profile_completion_show_about b on a.session_long=b.session_long and a.session=b.session
    left join profile_completion_action_about c on a.session_long=c.session_long and a.session=c.session
    left join profile_completion_show_name d on a.session_long=d.session_long and a.session=d.session
    left join profile_completion_action_name e on a.session_long=e.session_long and a.session=e.session
    left join profile_completion_show_picture f on a.session_long=f.session_long and a.session=f.session
    left join profile_completion_action_picture g on a.session_long=g.session_long and a.session=g.session
    left join profile_completion_show_phone h on a.session_long=h.session_long and a.session=h.session
    left join profile_completion_action_phone i on a.session_long=i.session_long and a.session=i.session
    left join profile_completion_show_email j on a.session_long=j.session_long and a.session=j.session
    left join profile_completion_action_email k on a.session_long=k.session_long and a.session=k.session
order by session_long
 



"""


In [122]:
df = pd.read_sql_query(query.replace('%', '%%'), engine)

In [123]:
df.head()

,session_long,session,login_method,show_about,continue_about,show_name,continue_name,show_picture,continue_picture,show_phone,continue_phone,show_email,continue_email
0,002ad2c98d064c9e,16c41a2631fx913952e2,phone,1,0,0,0,1,0,0,0,1,0
1,0c1005dd1e617b0d,16c436cd00ex2a61d5c7,phone,0,0,0,0,0,0,0,0,0,0
2,0de713ba5c4d5202,16c42123f7cx43ad5493,phone,0,0,0,0,0,0,0,0,0,0
3,10e60df767839802,16c42562175xaaa8b634,google,1,0,0,0,1,0,0,0,0,0
4,1338dd09f6120899,16c41c715a9x5f72083d,phone,1,0,0,0,1,0,0,0,1,0


In [124]:
df['show_about_2'] = df.groupby(['session' , 'login_method'])['show_about'].transform('max')
df['continue_about_2'] = df.groupby(['session' , 'login_method'])['continue_about'].transform('max')
df['show_name_2'] = df.groupby(['session' , 'login_method'])['show_name'].transform('max')
df['continue_name_2'] = df.groupby(['session' , 'login_method'])['continue_name'].transform('max')
df['show_picture_2'] = df.groupby(['session' , 'login_method'])['show_picture'].transform('max')
df['continue_picture_2'] = df.groupby(['session' , 'login_method'])['continue_picture'].transform('max')
df['show_phone_2'] = df.groupby(['session' ,'login_method'])['show_phone'].transform('max')
df['continue_phone_2'] = df.groupby(['session','login_method'])['continue_phone'].transform('max')
df['show_email_2'] = df.groupby(['session','login_method'])['show_email'].transform('max')
df['continue_email_2'] = df.groupby(['session','login_method'])['continue_email'].transform('max')

In [125]:
df.head()

,session_long,session,login_method,show_about,continue_about,show_name,continue_name,show_picture,continue_picture,show_phone,...,show_about_2,continue_about_2,show_name_2,continue_name_2,show_picture_2,continue_picture_2,show_phone_2,continue_phone_2,show_email_2,continue_email_2
0,002ad2c98d064c9e,16c41a2631fx913952e2,phone,1,0,0,0,1,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0c1005dd1e617b0d,16c436cd00ex2a61d5c7,phone,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0de713ba5c4d5202,16c42123f7cx43ad5493,phone,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10e60df767839802,16c42562175xaaa8b634,google,1,0,0,0,1,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1338dd09f6120899,16c41c715a9x5f72083d,phone,1,0,0,0,1,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [126]:
dfs = df[['session_long' , 'login_method', 'show_about_2' , 'continue_about_2' , 'show_name_2' , 'continue_name_2', 'show_picture_2' ,
          'continue_picture_2' ,'show_phone_2' ,'continue_phone_2' ,'show_email_2' ,'continue_email_2']]

In [127]:
dfs.head()

,session_long,login_method,show_about_2,continue_about_2,show_name_2,continue_name_2,show_picture_2,continue_picture_2,show_phone_2,continue_phone_2,show_email_2,continue_email_2
0,002ad2c98d064c9e,phone,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0c1005dd1e617b0d,phone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0de713ba5c4d5202,phone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10e60df767839802,google,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1338dd09f6120899,phone,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [128]:
dfs.groupby(['login_method'])['show_about_2' , 'continue_about_2' , 'show_name_2' , 'continue_name_2', 
                              'show_picture_2' ,'continue_picture_2' ,'show_phone_2' ,'continue_phone_2' ,
                              'show_email_2' ,'continue_email_2'].sum()


#dfs.pivot_table(index='login_method' , 
#               columns=['show_about_2' , 'continue_about_2' , 'show_name_2' , 'continue_name_2', 
#                         'show_picture_2' ,'continue_picture_2' ,'show_phone_2' ,'continue_phone_2' ,
#                         'show_email_2' ,'continue_email_2'] , 
#               values=['show_about_2' , 'continue_about_2' , 'show_name_2' , 'continue_name_2', 
#                       'show_picture_2' ,'continue_picture_2' ,'show_phone_2' ,'continue_phone_2' ,
#                       'show_email_2' ,'continue_email_2'] ,
#               aggfunc={sum})

,show_about_2,continue_about_2,show_name_2,continue_name_2,show_picture_2,continue_picture_2,show_phone_2,continue_phone_2,show_email_2,continue_email_2
login_method,,,,,,,,,,
,2.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,0.0,0.0
email,2439.0,309.0,331.0,280.0,3193.0,410.0,2537.0,976.0,4.0,0.0
facebook,21280.0,2553.0,518.0,406.0,17444.0,1592.0,25196.0,10250.0,21059.0,2956.0
google,40529.0,5995.0,3343.0,2628.0,20169.0,1460.0,43760.0,16531.0,312.0,36.0
phone,34381.0,2785.0,19229.0,16889.0,41159.0,5282.0,31.0,5.0,36065.0,6104.0


# There are more DAU navigating with information complete?

## Users of today

In [4]:
query = """

select
 split_part(user_sk , '|' , 6) as user_id
  from ods.fact_user_hydra_browsing_activity
 where is_panamera = true
  and country_sk = 'olx|asia|in'
  and date_nk = '2019-08-06'
  and user_sk != 'unknown'
  and app_version = '13.19.02'
group by 1
order by random()
limit 50000

"""


In [5]:
df_users_today = pd.read_sql_query(query.replace('%', '%%'), engine)

In [6]:
df_users_today.head()

,user_id
0,421926492
1,148701349
2,41260053
3,408895337
4,422656556


In [7]:
# Connection to the SQL database
import pymysql.cursors
connect_sql = pymysql.connect(host='prd-db-olxin-replica-1.cnhuszkczyjo.ap-southeast-1.rds.amazonaws.com',
                            user='dev',
                            password='^b6W2vVuM4KJ',
                            db='olxin',
                            charset='utf8mb4',
                            cursorclass=pymysql.cursors.DictCursor)

In [8]:
query = """

SELECT
        sum(case when ((google_id is null and facebook_id is null) or (google_id = '' and facebook_id = '')) then 0 else 1 end) social_account,
        sum(case when default_phone is null or default_phone='' then 0 else 1 end) as tiene_phone,
        sum(CASE
             WHEN email LIKE '%olx.co%' THEN 0 -- Autogenerado
             WHEN email LIKE '%@olx%' THEN 0 -- Autogenerado
             WHEN email LIKE '%DELETED%' THEN 0 -- Autogenerados
             WHEN email LIKE '%UNLINK%' THEN 0 -- Autogenerados
             ELSE 1 END)  as valido_email,
        sum(case when default_person!='OLX User' then 1 else 0 end) as real_name,
        sum(case when b.external_key is not Null and b.external_key>0 then 1 else 0 end) has_photo,
        sum(case when about != '' then 1 else 0 end) has_about,
        count(id)
        -- SUBSTRING_INDEX(email,'@',-1) as domain
FROM olxin.users a
    left join (select distinct external_key from olxin.images where active=1 and type='user') b on a.id=b.external_key
   where type in ('confirmed')
        and id in {} 

""".format('(' + ', '.join(pd.Series(df_users_today.user_id).astype('str').apply(lambda x: "'"  + x + "'")) + ')')

In [9]:
df_ora = pd.read_sql(query, con=connect_sql)   

/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:329: Warning: (3170, "Memory capacity of 8388608 bytes for 'range_optimizer_max_mem_size' exceeded. Range optimization was not done for this query.")
  self._do_get_result()


In [10]:
df_ora.head()

,social_account,tiene_phone,valido_email,real_name,has_photo,has_about,count(id)
0,34791.0,40634.0,26591.0,46063.0,17283.0,4251.0,49492


## Users of month 3 year = 2019

In [11]:
query = """

select
 split_part(user_sk , '|' , 6) as user_id
  from ods.fact_user_hydra_browsing_activity
 where is_panamera = true
  and country_sk = 'olx|asia|in'
  and user_sk != 'unknown'
group by 1
having max(date_nk) = '2019-03-28'
order by random()
limit 50000

"""


In [12]:
df_users_march = pd.read_sql_query(query.replace('%', '%%'), engine)

In [13]:
df_users_march.head()

,user_id
0,299243531
1,173293433
2,406180785
3,404391915
4,406167124


In [14]:
df_users_march.shape

(50000, 1)

In [15]:
query = """

SELECT
        sum(case when ((google_id is null and facebook_id is null) or (google_id = '' and facebook_id = '')) then 0 else 1 end) social_account,
        sum(case when default_phone is null or default_phone='' then 0 else 1 end) as tiene_phone,
        sum(CASE
             WHEN email LIKE '%olx.co%' THEN 0 -- Autogenerado
             WHEN email LIKE '%@olx%' THEN 0 -- Autogenerado
             WHEN email LIKE '%DELETED%' THEN 0 -- Autogenerados
             WHEN email LIKE '%UNLINK%' THEN 0 -- Autogenerados
             ELSE 1 END)  as valido_email,
        sum(case when default_person!='OLX User' then 1 else 0 end) as real_name,
        sum(case when b.external_key is not Null and b.external_key>0 then 1 else 0 end) has_photo,
        sum(case when about != '' then 1 else 0 end) has_about,
        count(id)
        -- SUBSTRING_INDEX(email,'@',-1) as domain
FROM olxin.users a
    left join (select distinct external_key from olxin.images where active=1 and type='user') b on a.id=b.external_key
   where type in ('confirmed')
        and id in {} 

""".format('(' + ', '.join(pd.Series(df_users_march.user_id).astype('str').apply(lambda x: "'"  + x + "'")) + ')')

In [16]:
df_march = pd.read_sql(query, con=connect_sql) 

In [17]:
df_march.head()

,social_account,tiene_phone,valido_email,real_name,has_photo,has_about,count(id)
0,28479.0,24578.0,19819.0,37652.0,14484.0,262.0,46354


## Building results

In [18]:
tot = df_ora.append(df_march)
tot['time'] = ['today' , 'last_day_march']
tot.head()

,social_account,tiene_phone,valido_email,real_name,has_photo,has_about,count(id),time
0,34791.0,40634.0,26591.0,46063.0,17283.0,4251.0,49492,today
0,28479.0,24578.0,19819.0,37652.0,14484.0,262.0,46354,last_day_march


In [19]:
tot['perc_socialaccount'] = tot['social_account'] /tot['count(id)']
tot['perc_tiene_phone'] = tot['tiene_phone'] /tot['count(id)']
tot['perc_valido_email'] = tot['valido_email'] /tot['count(id)']
tot['perc_real_name'] = tot['real_name'] /tot['count(id)']
tot['perc_has_photo'] = tot['has_photo'] /tot['count(id)']
tot['perc_has_about'] = tot['has_about'] /tot['count(id)']


In [20]:
tot2 = tot[['time' , 'perc_socialaccount' , 'perc_tiene_phone' , 'perc_valido_email' ,'perc_real_name',
           'perc_has_photo' , 'perc_has_about']]

In [21]:
tot2.melt(id_vars='time')
tot3 = tot2.set_index('time').T
tot3['uplift'] = (tot3['today'] / tot3['last_day_march']) - 1
tot3.head(20)

time,today,last_day_march,uplift
perc_socialaccount,0.702962,0.614381,0.144180
perc_tiene_phone,0.821022,0.530224,0.548443
perc_valido_email,0.537279,0.427557,0.256623
perc_real_name,0.930716,0.812271,0.145820
perc_has_photo,0.349208,0.312465,0.117591
perc_has_about,0.085893,0.005652,14.196446


In [22]:
#uplift_1 = tot3['uplift']
uplift_2 = tot3['uplift']